In [1]:
from openai import OpenAI

In [2]:
from tqdm.auto import tqdm

In [3]:
client = OpenAI()

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client=Elasticsearch("http://localhost:9200")

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name= "course_questions_v2"

es_client.indices.create(index=index_name, body= index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions_v2'})

In [24]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)    

  0%|          | 0/948 [00:00<?, ?it/s]

In [25]:
query = "How do I execute a command in a running docker container?"

In [26]:
search_query_filter = {
    "size": 4,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [27]:
search_query = {
    "size": 4,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },            
        }
    }
}

In [28]:
search_query

{'size': 4,
 'query': {'bool': {'must': {'multi_match': {'query': 'How do I execute a command in a running docker container?',
     'fields': ['question^4', 'text'],
     'type': 'best_fields'}}}}}

In [29]:
response = es_client.search(index=index_name, body=search_query)

In [30]:
result_docs =[]
result_scores=[]
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    result_scores.append(hit['_score'])
print(result_scores)
print (max(result_scores))
    

[84.050095, 75.54128, 72.08518, 51.04628]
84.050095


In [31]:
response = es_client.search(index=index_name, body=search_query_filter)

In [32]:
result_docs =[]
result_q =[]
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])   
    result_q.append(hit['_source']['question']) 
print(result_docs)
print(result_q[2])

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'm

In [33]:
context_template = """
Q: {question}
A: {text}
""".strip()

final_context=""

for doc in result_docs:
    context = context_template.format(question=doc["question"], text=doc["text"]).strip()
    final_context = final_context + context + "\n\n"

final_context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [34]:
query = "How do I execute a command in a running docker container?"

In [35]:
def build_prompt(query, context):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()  
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
p = build_prompt (query, final_context)
print (p)
len(p)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

2061

In [37]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [39]:
tokens = encoding.encode(p)

In [40]:
num_tokens = len(tokens)
num_tokens

455